# Stock Tweets Sentiment Analysis with VADER

This notebook performs sentiment analysis on stock-related tweets using VADER (Valence Aware Dictionary and sEntiment Reasoner).

**VADER** is specifically designed for social media text and works well with:
- Emojis
- Slang
- Abbreviations
- Capitalization (for emphasis)

**Sentiment Scores:**
- `positive`: Positive sentiment score (0-1)
- `negative`: Negative sentiment score (0-1)
- `neutral`: Neutral sentiment score (0-1)
- `compound`: Overall sentiment (-1 to +1)
  - **compound >= 0.05**: Positive
  - **compound <= -0.05**: Negative
  - **-0.05 < compound < 0.05**: Neutral

## Step 1: Import Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## Step 2: Load the Data

In [ ]:
# Load the cleaned stock tweets data
df = pd.read_csv('Cleaned_Stock_Tweets.csv')

# Display basic information
print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Basic statistics
print(f"\nTotal Tweets: {len(df):,}")
print(f"Unique Stocks: {df['Stock Name'].nunique()}")
print(f"\nTop 5 Stocks by Tweet Count:")
print(df['Stock Name'].value_counts().head())

## Step 3: Initialize VADER Sentiment Analyzer

In [ ]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Test VADER on a sample tweet
sample_tweet = df['Tweet'].iloc[0]
sample_scores = analyzer.polarity_scores(sample_tweet)

print("Sample Tweet:")
print(f"{sample_tweet}\n")
print("Sample Sentiment Scores:")
print(sample_scores)

## Step 4: Apply Sentiment Analysis to All Tweets

In [ ]:
# Function to get sentiment scores
def get_sentiment_scores(text):
    """
    Analyzes sentiment of text using VADER.
    Returns a dictionary with sentiment scores.
    """
    if pd.isna(text):
        return {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0}
    return analyzer.polarity_scores(str(text))

# Apply sentiment analysis (this may take a minute)
print("Analyzing sentiment for all tweets...")
print("This may take a minute depending on dataset size...\n")

# Get sentiment scores for each tweet
sentiment_scores = df['Tweet'].apply(get_sentiment_scores)

# Create separate columns for each score
df['sentiment_negative'] = sentiment_scores.apply(lambda x: x['neg'])
df['sentiment_neutral'] = sentiment_scores.apply(lambda x: x['neu'])
df['sentiment_positive'] = sentiment_scores.apply(lambda x: x['pos'])
df['sentiment_compound'] = sentiment_scores.apply(lambda x: x['compound'])

print("✅ Sentiment analysis complete!")
print(f"\nProcessed {len(df):,} tweets")

# Display sample results
df[['Tweet', 'sentiment_negative', 'sentiment_neutral', 'sentiment_positive', 'sentiment_compound']].head(10)

## Step 5: Classify Sentiment as Positive/Negative/Neutral

In [ ]:
# Function to classify sentiment based on compound score
def classify_sentiment(compound_score):
    """
    Classifies sentiment based on compound score:
    - Positive: compound >= 0.05
    - Negative: compound <= -0.05
    - Neutral: -0.05 < compound < 0.05
    """
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply classification
df['sentiment_label'] = df['sentiment_compound'].apply(classify_sentiment)

# Display distribution
print("Sentiment Distribution:")
print(df['sentiment_label'].value_counts())
print(f"\nPercentages:")
print(df['sentiment_label'].value_counts(normalize=True) * 100)

## Step 6: Visualize Overall Sentiment Distribution

In [ ]:
# Create a pie chart of sentiment distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart
sentiment_counts = df['sentiment_label'].value_counts()
colors = ['#2ecc71', '#e74c3c', '#95a5a6']  # Green, Red, Gray
ax1.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax1.set_title('Overall Sentiment Distribution', fontsize=14, fontweight='bold')

# Bar chart
sentiment_counts.plot(kind='bar', ax=ax2, color=colors)
ax2.set_title('Sentiment Counts', fontsize=14, fontweight='bold')
ax2.set_xlabel('Sentiment', fontsize=12)
ax2.set_ylabel('Number of Tweets', fontsize=12)
ax2.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## Step 7: Analyze Sentiment by Stock

In [ ]:
# Calculate average sentiment by stock
stock_sentiment = df.groupby('Stock Name').agg({
    'sentiment_compound': 'mean',
    'Tweet': 'count'
}).rename(columns={'Tweet': 'tweet_count'})

stock_sentiment = stock_sentiment.sort_values('sentiment_compound', ascending=False)

print("Average Sentiment by Stock (Top 10):")
print(stock_sentiment.head(10))
print("\nBottom 10:")
print(stock_sentiment.tail(10))

In [ ]:
# Visualize top 10 stocks by average sentiment
top_10_stocks = stock_sentiment.head(10)

plt.figure(figsize=(12, 6))
colors_map = ['#2ecc71' if x >= 0 else '#e74c3c' for x in top_10_stocks['sentiment_compound']]
plt.barh(top_10_stocks.index, top_10_stocks['sentiment_compound'], color=colors_map)
plt.xlabel('Average Sentiment (Compound Score)', fontsize=12)
plt.ylabel('Stock', fontsize=12)
plt.title('Top 10 Stocks by Average Sentiment', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

## Step 8: Sentiment Distribution by Stock

In [ ]:
# Get sentiment distribution for top stocks (by tweet count)
top_stocks = df['Stock Name'].value_counts().head(5).index

# Create a stacked bar chart
sentiment_by_stock = pd.crosstab(df[df['Stock Name'].isin(top_stocks)]['Stock Name'], 
                                  df[df['Stock Name'].isin(top_stocks)]['sentiment_label'],
                                  normalize='index') * 100

sentiment_by_stock[['Positive', 'Neutral', 'Negative']].plot(kind='bar', 
                                                               stacked=True,
                                                               color=['#2ecc71', '#95a5a6', '#e74c3c'],
                                                               figsize=(12, 6))
plt.title('Sentiment Distribution for Top 5 Stocks (by Tweet Count)', fontsize=14, fontweight='bold')
plt.xlabel('Stock', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.legend(title='Sentiment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 9: Analyze Sentiment Over Time

In [ ]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Calculate daily average sentiment
daily_sentiment = df.groupby(df['Date'].dt.date)['sentiment_compound'].mean()

# Plot sentiment over time
plt.figure(figsize=(14, 6))
plt.plot(daily_sentiment.index, daily_sentiment.values, linewidth=2, color='#3498db')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
plt.fill_between(daily_sentiment.index, daily_sentiment.values, 0, 
                 where=(daily_sentiment.values >= 0), alpha=0.3, color='#2ecc71', label='Positive')
plt.fill_between(daily_sentiment.index, daily_sentiment.values, 0, 
                 where=(daily_sentiment.values < 0), alpha=0.3, color='#e74c3c', label='Negative')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average Sentiment (Compound Score)', fontsize=12)
plt.title('Average Sentiment Over Time', fontsize=14, fontweight='bold')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Step 10: Most Positive and Negative Tweets

In [ ]:
# Get most positive tweets
print("🟢 TOP 5 MOST POSITIVE TWEETS:")
print("=" * 80)
most_positive = df.nlargest(5, 'sentiment_compound')[['Tweet', 'Stock Name', 'sentiment_compound']]
for idx, row in most_positive.iterrows():
    print(f"\nStock: {row['Stock Name']} | Score: {row['sentiment_compound']:.3f}")
    print(f"Tweet: {row['Tweet'][:200]}...")
    print("-" * 80)

In [ ]:
# Get most negative tweets
print("🔴 TOP 5 MOST NEGATIVE TWEETS:")
print("=" * 80)
most_negative = df.nsmallest(5, 'sentiment_compound')[['Tweet', 'Stock Name', 'sentiment_compound']]
for idx, row in most_negative.iterrows():
    print(f"\nStock: {row['Stock Name']} | Score: {row['sentiment_compound']:.3f}")
    print(f"Tweet: {row['Tweet'][:200]}...")
    print("-" * 80)

## Step 11: Summary Statistics

In [ ]:
# Calculate summary statistics
print("📊 SENTIMENT ANALYSIS SUMMARY")
print("=" * 60)
print(f"\nTotal Tweets Analyzed: {len(df):,}")
print(f"Date Range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"Number of Unique Stocks: {df['Stock Name'].nunique()}")

print(f"\n📈 Overall Sentiment Statistics:")
print(f"  Average Compound Score: {df['sentiment_compound'].mean():.3f}")
print(f"  Median Compound Score: {df['sentiment_compound'].median():.3f}")
print(f"  Std Deviation: {df['sentiment_compound'].std():.3f}")

print(f"\n🎯 Sentiment Breakdown:")
sentiment_pct = df['sentiment_label'].value_counts(normalize=True) * 100
for label, pct in sentiment_pct.items():
    print(f"  {label}: {pct:.1f}%")

print(f"\n⭐ Most Discussed Stocks:")
top_3_stocks = df['Stock Name'].value_counts().head(3)
for stock, count in top_3_stocks.items():
    avg_sentiment = df[df['Stock Name'] == stock]['sentiment_compound'].mean()
    print(f"  {stock}: {count:,} tweets | Avg Sentiment: {avg_sentiment:.3f}")

## Step 12: Save Results to CSV

In [ ]:
# Save the dataframe with sentiment scores to a new CSV file
output_filename = 'Stock_Tweets_With_Sentiment.csv'
df.to_csv(output_filename, index=False)

print(f"✅ Results saved to: {output_filename}")
print(f"\nNew columns added:")
print("  - sentiment_negative")
print("  - sentiment_neutral")
print("  - sentiment_positive")
print("  - sentiment_compound")
print("  - sentiment_label (Positive/Negative/Neutral)")

## Step 13: Export Summary Report

In [ ]:
# Create a summary report by stock
summary_report = df.groupby('Stock Name').agg({
    'Tweet': 'count',
    'sentiment_compound': ['mean', 'std', 'min', 'max'],
    'sentiment_positive': 'mean',
    'sentiment_negative': 'mean',
    'sentiment_neutral': 'mean'
}).round(3)

# Flatten column names
summary_report.columns = ['_'.join(col).strip() for col in summary_report.columns.values]
summary_report = summary_report.rename(columns={'Tweet_count': 'tweet_count'})

# Add sentiment label counts
sentiment_counts = df.groupby(['Stock Name', 'sentiment_label']).size().unstack(fill_value=0)
summary_report = summary_report.join(sentiment_counts)

# Sort by tweet count
summary_report = summary_report.sort_values('tweet_count', ascending=False)

# Save summary report
summary_filename = 'Sentiment_Summary_By_Stock.csv'
summary_report.to_csv(summary_filename)

print(f"✅ Summary report saved to: {summary_filename}")
print(f"\nPreview of summary report:")
summary_report.head(10)

## 🎉 Analysis Complete!

### What You've Done:
1. ✅ Loaded and explored your stock tweets dataset
2. ✅ Applied VADER sentiment analysis to all tweets
3. ✅ Created visualizations of sentiment distributions
4. ✅ Analyzed sentiment by stock and over time
5. ✅ Identified most positive and negative tweets
6. ✅ Saved results to CSV files

### Output Files:
- `Stock_Tweets_With_Sentiment.csv` - Full dataset with sentiment scores
- `Sentiment_Summary_By_Stock.csv` - Summary statistics by stock

### Next Steps (Optional):
- Compare sentiment with actual stock price movements
- Analyze correlation between tweet volume and sentiment
- Build a sentiment trend predictor
- Filter by specific date ranges or stocks for deeper analysis